In [ ]:
from load_data import load_standardized_vitals, load_test_results_symptoms_sex_age

rhr_metric = load_standardized_vitals(
    "rhr",
).set_index(["user_id", "test_week_start"])

steps_metric = load_standardized_vitals(
    "steps",
).set_index(["user_id", "test_week_start"])

test_results = load_test_results_symptoms_sex_age().set_index(
    ["user_id", "test_week_start"]
)

In [ ]:
import plotly.express as px
import pandas as pd


def count_vitals(vital_type: str, df: pd.DataFrame) -> pd.DataFrame:
    tmp = (
        df.reset_index()
        .dropna()
        .groupby("test_week_start")[vital_type]
        .count()
        .reset_index()
    )
    tmp["test_week_start"] = tmp["test_week_start"].apply(pd.to_datetime)
    tmp = tmp.rename(columns={vital_type: "vitals"})
    print(tmp.columns)
    tmp["type"] = vital_type
    return tmp


rhr = count_vitals("rhr_metric", rhr_metric)
steps = count_vitals("steps_metric", steps_metric)
symptoms = count_vitals("cough", test_results)

all_vital_counts = pd.concat([rhr, steps, symptoms]).query("test_week_start > 20211001")

px.line(all_vital_counts, x="test_week_start", y="vitals", color="type").show()

In [ ]:
from psycopg2.sql import SQL
from load_data import pull_from_postgres

df = pull_from_postgres(
    SQL(
        """
    SELECT created_at FROM datenspende.vitaldata WHERE type = 9;
    """
    )
)
df

In [ ]:
test_results

In [ ]:
from load_data import pull_from_postgres
from psycopg2.sql import SQL

VITAL_TYPE = 9

vital_features = (
    pull_from_postgres(
        SQL(
            """
        SELECT
            *
        FROM
            jakob.ml_vital_features;
        """
        )
    )
    .groupby(["test_week_start", "type"])
    .count()
    .reset_index()
)

In [ ]:
vital_features

In [ ]:
px.line(vital_features, x="test_week_start", y="signal", color="type").show()